In [3]:
!pip install  fastf1 -q

In [4]:
YEAR = 2023

In [5]:
import fastf1
fastf1.Cache.enable_cache("cache")

In [6]:
import utils
import fastf1
def events_available(year: int) -> any:
    # get events available for a given year
    data = utils.LatestData(year)
    events = data.get_events()
    return events

events = events_available(YEAR)
print(events)

['Pre-Season Testing', 'Bahrain Grand Prix', 'Saudi Arabian Grand Prix', 'Australian Grand Prix', 'Azerbaijan Grand Prix', 'Miami Grand Prix', 'Monaco Grand Prix', 'Spanish Grand Prix', 'Canadian Grand Prix', 'Austrian Grand Prix', 'British Grand Prix', 'Hungarian Grand Prix', 'Belgian Grand Prix', 'Dutch Grand Prix', 'Italian Grand Prix', 'Singapore Grand Prix', 'Japanese Grand Prix', 'Qatar Grand Prix', 'United States Grand Prix', 'Mexico City Grand Prix', 'São Paulo Grand Prix', 'Las Vegas Grand Prix', 'Abu Dhabi Grand Prix']


In [7]:
def sessions_available(year: int, event: str | int) -> any:
    # get sessions available for a given year and event
    event = str(event)
    data = utils.LatestData(year)
    sessions = data.get_sessions(event)
    return sessions



In [8]:
# import os

# # Your list of events
# events_list = events

# # Create a folder for each event in the current working directory
# for event in events_list:
#     event_directory = os.path.join(os.getcwd(), event)

#     # Check if the folder already exists before creating it
#     if not os.path.exists(event_directory):
#         os.makedirs(event_directory)
#         print(f"Folder created for {event}: {event_directory}")
#     else:
#         print(f"Folder already exists for {event}: {event_directory}")







In [9]:
def session_drivers(year: int, event: str | int, session: str) -> any:
    # get drivers available for a given year, event and session
    import fastf1
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=True, weather=False, messages=False)

    laps = f1session.laps
    team_colors = utils.team_colors(year)
    # add team_colors dict to laps on Team column
    laps["color"] = laps["Team"].map(team_colors)

    unique_drivers = laps["Driver"].unique()

    drivers = [
        {

            "driver": driver,
            "team": laps[laps.Driver == driver].Team.iloc[0],
        }
        for driver in unique_drivers
    ]

    return  {"drivers": drivers}

In [10]:
# import os

# # Your list of events
# events_list = events

# # Loop through each event
# for event in events_list:
#     event_directory = os.path.join(os.getcwd(), event)

#     # Check if the event folder already exists before creating it
#     if not os.path.exists(event_directory):
#         os.makedirs(event_directory)
#         print(f"Folder created for {event}: {event_directory}")
#     else:
#         print(f"Folder already exists for {event}: {event_directory}")

#     # Get sessions for the current event
#     if event == "Qatar Grand Prix":
#         sessions = ['Practice 1', 'Qualifying', 'Sprint Shootout', 'Sprint', 'Race']
#     else:
#         sessions = sessions_available(YEAR, event)


#     # Loop through each session and create a folder within the event folder
#     for session in sessions:
#         session_directory = os.path.join(event_directory, session)

#         # Check if the session folder already exists before creating it
#         if not os.path.exists(session_directory):
#             os.makedirs(session_directory)
#             print(f"Folder created for {event} - {session}: {session_directory}")
#         else:
#             print(f"Folder already exists for {event} - {session}: {session_directory}")



In [11]:
# import os

# # Your list of events
# events_list = events

# # Loop through each event
# for event in events_list:


#     # Get sessions for the current event
#     if event == "Qatar Grand Prix":
#         sessions = ['Practice 1', 'Qualifying', 'Sprint Shootout', 'Sprint', 'Race']
#     else:
#         sessions = sessions_available(YEAR, event)

#     # Loop through each session and create a folder within the event folder
#     for session in sessions:
#         drivers = session_drivers(YEAR, event, session)

#         import json

#         # Specify the file path where you want to save the JSON data
#         file_path = f"{event}/{session}/drivers.json"

#         # Save the dictionary to a JSON file
#         with open(file_path, "w") as json_file:
#             json.dump(drivers, json_file)

#         print(f"Dictionary saved to {file_path}")


In [12]:
def session_drivers_list(year: int, event: str | int, session: str) -> any:
    # get drivers available for a given year, event and session
    import fastf1
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=True, weather=False, messages=False)

    laps = f1session.laps


    unique_drivers = laps["Driver"].unique()


    return  list(unique_drivers)

In [17]:
def laps_data(year: int, event: str | int, session: str, driver: str) -> any:
    # get drivers available for a given year, event, and session
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps

    # add team_colors dict to laps on Team column

    # for each driver in drivers, get the Team column from laps and get the color from team_colors dict
    drivers_data = []

    driver_laps = laps.pick_driver(driver)
    driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()
    # remove rows where LapTime is null
    driver_laps = driver_laps[driver_laps.LapTime.notnull()]

    drivers_data = {
        "time": driver_laps["LapTime"].tolist(),
        "lap": driver_laps["LapNumber"].tolist(),
        "compound": driver_laps["Compound"].tolist(),
    }



    return drivers_data


# # Example usage:
# result = laps_data(2018, "Bahrain", "R", "GAS")
# result

In [15]:
# # reversing a pyton list
# events.reverse()

In [18]:
import os
import json

# Your list of events
events_list = events

# Loop through each event
for event in events_list:

    # Get sessions for the current event
    if event == "Qatar Grand Prix":
        sessions = ['Practice 1', 'Qualifying', 'Sprint Shootout', 'Sprint', 'Race']
    else:
        sessions = sessions_available(YEAR, event)

    # Loop through each session and create a folder within the event folder
    for session in sessions:
        drivers = session_drivers_list(YEAR, event, session)

        for driver in drivers:
            # Create a folder for the driver if it doesn't exist
            driver_folder = f"{event}/{session}/{driver}"
            if not os.path.exists(driver_folder):
                os.makedirs(driver_folder)

            laptimes = laps_data(YEAR, event, session, driver)

            # Specify the file path where you want to save the JSON data
            file_path = f"{driver_folder}/laptimes.json"

            # Save the dictionary to a JSON file
            with open(file_path, "w") as json_file:
                json.dump(laptimes, json_file)

            # print(f"Dictionary saved to {file_path}")




core           INFO 	Loading data for Qatar Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '31', '4', '40', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Qatar Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for 

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['81', '1', '4', '63', '44', '55', '23', '14', '10', '77', '22', '16', '18', '20', '24', '27', '31', '11', '2', '40']
core           INFO 	Loading data for Qatar Grand Prix - Sprint [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req  

In [23]:
import numpy as np

def accCalc(allLapsDriverTelemetry, Nax, Nay, Naz):
    vx = allLapsDriverTelemetry['Speed'] / 3.6
    time_float = allLapsDriverTelemetry['Time'] / np.timedelta64(1, 's')
    dtime = np.gradient(time_float)
    ax = np.gradient(vx) / dtime

    for i in np.arange(1, len(ax) - 1).astype(int):
        if ax[i] > 25:
            ax[i] = ax[i - 1]

    ax_smooth = np.convolve(ax, np.ones((Nax,)) / Nax, mode='same')
    x = allLapsDriverTelemetry['X']
    y = allLapsDriverTelemetry['Y']
    z = allLapsDriverTelemetry['Z']

    dx = np.gradient(x)
    dy = np.gradient(y)
    dz = np.gradient(z)

    theta = np.arctan2(dy, (dx + np.finfo(float).eps))
    theta[0] = theta[1]
    theta_noDiscont = np.unwrap(theta)

    dist = allLapsDriverTelemetry['Distance']
    ds = np.gradient(dist)
    dtheta = np.gradient(theta_noDiscont)

    for i in np.arange(1, len(dtheta) - 1).astype(int):
        if abs(dtheta[i]) > 0.5:
            dtheta[i] = dtheta[i - 1]

    C = dtheta / (ds + 0.0001)  # To avoid division by 0

    ay = np.square(vx) * C
    indexProblems = np.abs(ay) > 150
    ay[indexProblems] = 0

    ay_smooth = np.convolve(ay, np.ones((Nay,)) / Nay, mode='same')

    # for z
    z_theta = np.arctan2(dz, (dx + np.finfo(float).eps))
    z_theta[0] = z_theta[1]
    z_theta_noDiscont = np.unwrap(z_theta)

    dist = allLapsDriverTelemetry['Distance']
    ds = np.gradient(dist)
    z_dtheta = np.gradient(z_theta_noDiscont)

    for i in np.arange(1, len(z_dtheta) - 1).astype(int):
        if abs(z_dtheta[i]) > 0.5:
            z_dtheta[i] = z_dtheta[i - 1]

    z_C = z_dtheta / (ds + 0.0001)  # To avoid division by 0

    az = np.square(vx) * z_C
    indexProblems = np.abs(az) > 150
    az[indexProblems] = 0

    az_smooth = np.convolve(az, np.ones((Naz,)) / Naz, mode='same')



    allLapsDriverTelemetry['Ax'] = ax_smooth
    allLapsDriverTelemetry['Ay'] = ay_smooth
    allLapsDriverTelemetry['Az'] = az_smooth

    return allLapsDriverTelemetry


In [34]:
def telemetry_data(year, event, session: str, driver, lap_number):
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=True, weather=False, messages=False)
    laps = f1session.laps

    driver_laps = laps.pick_driver(driver)
    driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()

    # get the telemetry for lap_number
    selected_lap = driver_laps[driver_laps.LapNumber == lap_number]

    telemetry = selected_lap.get_telemetry()

    acc_tel = accCalc(telemetry, 3, 9, 9)

    acc_tel["Time"] = acc_tel["Time"].dt.total_seconds()

    laptime = selected_lap.LapTime.values[0]
    data_key = f"{driver} - Lap {int(lap_number)} - {year} - {session} - [{laptime}]"

    acc_tel["DRS"] = acc_tel["DRS"].apply(lambda x: 1 if x in [10, 12, 14] else 0)
    acc_tel["Brake"] = acc_tel["Brake"].apply(lambda x: 1 if x == "True" else 0)

    telemetry_data = {"tel":{
        "time": acc_tel["Time"].tolist(),
        "rpm": acc_tel["RPM"].tolist(),
        "speed": acc_tel["Speed"].tolist(),
        "gear": acc_tel["nGear"].tolist(),
        "throttle": acc_tel["Brake"].tolist(),
        "brake": acc_tel["Distance"].tolist(),
        "drs": acc_tel["DRS"].tolist(),
        "distance": acc_tel["Distance"].tolist(),
        "rel_distance": acc_tel["RelativeDistance"].tolist(),
        "acc_x": acc_tel["Ax"].tolist(),
        "acc_y": acc_tel["Ay"].tolist(),
        "acc_z": acc_tel["Az"].tolist(),
        "x": acc_tel["X"].tolist(),
        "y": acc_tel["Y"].tolist(),
        "z": acc_tel["Z"].tolist(),
        "dataKey": data_key,
        }
    }
    return telemetry_data

In [39]:
import os
import json

# Your list of events
events_list = events

# Loop through each event
for event in events_list:

    # Get sessions for the current event
    if event == "Qatar Grand Prix":
        sessions = ['Practice 1', 'Qualifying', 'Sprint Shootout', 'Sprint', 'Race']
    else:
        sessions = sessions_available(YEAR, event)

    # Loop through each session and create a folder within the event folder
    for session in sessions:
        drivers = session_drivers_list(YEAR, event, session)

        for driver in drivers:
            f1session = fastf1.get_session(year,event, session)
            f1session.load(telemetry=False, weather=False, messages=False)
            laps = f1session.laps
            driver_laps = laps.pick_driver(driver)
            driver_lap_numbers = round(driver_laps["LapNumber"]).tolist()

            for lap_number in driver_lap_numbers:
                driver_folder = f"{event}/{session}/{driver}"
                if not os.path.exists(driver_folder):
                    os.makedirs(driver_folder)

                telemetry = telemetry_data(year, event, session, driver, lap_number)

                print(telemetry)

                # Specify the file path where you want to save the JSON data
                file_path = f"{driver_folder}/{lap_number}_tel.json"

                # Save the dictionary to a JSON file
                with open(file_path, "w") as json_file:
                    json.dump(telemetry, json_file)




core           INFO 	Loading data for British Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']
core           INFO 	Loading data for British Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']
core           INFO 	Loading data for British Grand 

{'tel': {'time': [0.0, 0.008, 0.17, 0.288, 0.31, 0.488, 0.63, 0.688, 0.81, 0.888, 1.128, 1.25, 1.408, 1.47, 1.648, 1.77, 1.848, 2.048, 2.069, 2.208, 2.249, 2.368, 2.449, 2.528, 2.709, 2.808, 2.869, 2.968, 3.128, 3.129, 3.368, 3.369, 3.549, 3.608, 3.709, 3.808, 3.869, 3.968, 4.029, 4.309, 4.328, 4.488, 4.509, 4.669, 4.728, 4.929, 5.088, 5.189, 5.248, 5.549, 5.728, 5.969, 6.168, 6.45, 6.529, 6.55, 6.688, 6.85, 7.01, 7.168, 7.31, 7.368, 7.51, 7.528, 7.688, 7.77, 7.888, 8.05, 8.168, 8.29, 8.368, 8.47, 8.608, 8.61, 8.869, 9.008, 9.209, 9.368, 9.549, 9.569, 9.608, 9.808, 9.969, 10.149, 10.248, 10.329, 10.429, 10.528, 10.728, 10.809, 10.968, 11.189, 11.248, 11.389, 11.488, 11.529, 11.688, 11.689, 11.928, 12.029, 12.168, 12.249, 12.409, 12.509, 12.528, 12.809, 12.869, 13.049, 13.13, 13.209, 13.39, 13.488, 13.53, 13.688, 13.83, 13.928, 13.99, 14.15, 14.368, 14.47, 14.55, 14.648, 14.85, 14.968, 15.09, 15.289, 15.328, 15.528, 15.629, 15.889, 15.968, 16.149, 16.328, 16.329, 16.688, 16.709, 16.968,

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']
/tmp/ipykernel_4161/1207580307.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()
core           INFO 	Loading data for British Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req   

{'tel': {'time': [0.0, 0.086, 0.246, 0.465, 0.506, 0.745, 0.905, 0.965, 1.065, 1.145, 1.304, 1.345, 1.525, 1.704, 1.805, 1.904, 1.985, 2.185, 2.304, 2.425, 2.544, 2.744, 2.865, 3.085, 3.104, 3.405, 3.464, 3.624, 3.625, 3.845, 3.904, 4.005, 4.064, 4.225, 4.344, 4.565, 4.704, 4.885, 5.025, 5.106, 5.225, 5.326, 5.425, 5.566, 5.865, 5.966, 6.185, 6.446, 6.465, 6.705, 6.726, 6.905, 6.986, 7.186, 7.305, 7.485, 7.665, 7.685, 7.785, 7.985, 8.064, 8.185, 8.224, 8.345, 8.504, 8.565, 8.744, 8.885, 9.145, 9.184, 9.445, 9.504, 9.824, 9.945, 10.024, 10.145, 10.424, 10.485, 10.744, 10.745, 10.865, 11.024, 11.165, 11.184, 11.424, 11.445, 11.626, 11.786, 11.825, 11.946, 12.025, 12.146, 12.185, 12.406, 12.505, 12.566, 12.745, 12.826, 13.025, 13.066, 13.246, 13.345, 13.526, 13.545, 13.686, 13.785, 13.925, 14.005, 14.185, 14.265, 14.465, 14.545, 14.665, 14.825, 15.225, 15.464, 15.565, 15.704, 15.825, 15.864, 16.045, 16.064, 16.325, 16.464, 16.505, 16.704, 16.745, 16.945, 17.104, 17.165, 17.304, 17.464, 17

...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']
/tmp/ipykernel_4161/1207580307.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()
core           INFO 	Loading data for British Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req   

{'tel': {'time': [0.0, 0.215, 0.373, 0.495, 0.513, 0.735, 0.913, 0.935, 1.273, 1.375, 1.535, 1.613, 1.895, 1.913, 2.054, 2.173, 2.294, 2.413, 2.494, 2.573, 2.694, 2.953, 2.974, 3.253, 3.433, 3.494, 3.573, 3.773, 3.894, 3.953, 4.175, 4.233, 4.454, 4.495, 4.654, 4.874, 4.895, 5.174, 5.375, 5.494, 5.655, 5.734, 5.975, 5.994, 6.215, 6.353, 6.575, 6.693, 6.855, 6.873, 7.055, 7.073, 7.313, 7.415, 7.473, 7.575, 7.693, 7.993, 8.015, 8.173, 8.353, 8.415, 8.553, 8.615, 8.653, 8.894, 9.053, 9.094, 9.413, 9.414, 9.614, 9.653, 9.873, 9.934, 10.033, 10.293, 10.334, 10.573, 10.775, 10.833, 11.014, 11.095, 11.194, 11.375, 11.454, 11.534, 11.615, 11.754, 11.775, 11.935, 11.954, 12.135, 12.294, 12.454, 12.455, 12.594, 12.695, 12.774, 13.015, 13.053, 13.175, 13.233, 13.375, 13.533, 13.735, 13.753, 13.953, 14.095, 14.293, 14.335, 14.673, 14.695, 14.975, 15.033, 15.175, 15.293, 15.335, 15.494, 15.513, 15.694, 15.833, 15.894, 15.993, 16.094, 16.233, 16.254, 16.413, 16.494, 16.613, 16.694, 16.853, 16.854, 17

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']
/tmp/ipykernel_4161/1207580307.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()
core           INFO 	Loading data for British Grand Prix - Practice 1 [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req   

{'tel': {'time': [0.0, 0.033, 0.075, 0.355, 0.433, 0.515, 0.693, 0.755, 0.793, 0.955, 1.093, 1.115, 1.313, 1.395, 1.555, 1.713, 1.755, 1.913, 2.075, 2.253, 2.315, 2.475, 2.493, 2.733, 2.833, 2.875, 3.053, 3.075, 3.213, 3.275, 3.435, 3.553, 3.733, 3.755, 3.953, 4.115, 4.254, 4.355, 4.434, 4.515, 4.734, 4.795, 5.014, 5.075, 5.275, 5.294, 5.534, 5.635, 5.734, 5.795, 5.934, 6.075, 6.114, 6.293, 6.355, 6.553, 6.555, 6.715, 6.773, 7.035, 7.053, 7.273, 7.395, 7.533, 7.635, 7.853, 7.875, 8.133, 8.235, 8.373, 8.395, 8.715, 8.733, 8.875, 8.953, 9.115, 9.373, 9.435, 9.553, 9.793, 9.833, 9.915, 10.075, 10.193, 10.355, 10.533, 10.556, 10.734, 10.755, 10.974, 11.115, 11.334, 11.595, 11.614, 11.755, 11.874, 11.894, 12.035, 12.194, 12.354, 12.435, 12.614, 12.635, 12.835, 13.033, 13.235, 13.313, 13.513, 13.675, 13.733, 13.915, 13.973, 14.235, 14.353, 14.435, 14.595, 14.753, 14.955, 15.073, 15.273, 15.275, 15.475, 15.635, 15.773, 15.915, 16.173, 16.195, 16.413, 16.475, 16.613, 16.813, 16.915, 17.113, 17

req            INFO 	Using cached data for car_data


In [29]:
year = YEAR
event = "Bahrain Grand Prix"
session = "Race"


In [36]:
import json
import os
drivers = session_drivers_list(year,event, session)
for driver in drivers:
    f1session = fastf1.get_session(year,event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps
    driver_laps = laps.pick_driver(driver)
    driver_lap_numbers = driver_laps["LapNumber"].tolist()
    for lap_number in driver_lap_numbers:
        driver_folder = f"{event}/{session}/{driver}"
        if not os.path.exists(driver_folder):
            os.makedirs(driver_folder)

        telemetry = telemetry_data(year, event, session, driver, lap_number)

        print(telemetry)

        # Specify the file path where you want to save the JSON data
        file_path = f"{driver_folder}/{lap_number}_tel.json"

        # Save the dictionary to a JSON file
        with open(file_path, "w") as json_file:
            json.dump(telemetry, json_file)







core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core 

{'tel': {'time': [0.0, 0.11, 0.185, 0.31, 0.445, 0.545, 0.59, 0.764, 0.95, 1.124, 1.19, 1.325, 1.35, 1.55, 1.585, 1.83, 1.965, 1.99, 2.19, 2.385, 2.51, 2.585, 2.75, 2.765, 2.91, 3.025, 3.15, 3.185, 3.385, 3.47, 3.63, 3.645, 3.79, 4.029, 4.045, 4.225, 4.27, 4.405, 4.63, 4.705, 4.83, 4.865, 5.07, 5.085, 5.325, 5.35, 5.55, 5.745, 6.03, 6.105, 6.19, 6.265, 6.51, 6.565, 6.645, 6.79, 6.925, 7.085, 7.15, 7.404, 7.47, 7.804, 7.87, 7.985, 8.11, 8.245, 8.345, 8.51, 8.665, 8.67, 8.885, 8.95, 8.985, 9.11, 9.185, 9.51, 9.525, 9.665, 9.71, 9.925, 9.95, 10.19, 10.305, 10.39, 10.565, 10.83, 10.965, 11.065, 11.11, 11.265, 11.31, 11.51, 11.545, 11.705, 11.75, 12.005, 12.07, 12.365, 12.47, 12.585, 12.75, 12.925, 13.07, 13.205, 13.47, 13.71, 13.725, 13.87, 14.03, 14.124, 14.23, 14.404, 14.43, 14.67, 14.785, 14.91, 15.065, 15.11, 15.245, 15.27, 15.55, 15.585, 15.765, 15.95, 15.985, 16.11, 16.205, 16.305, 16.35, 16.51, 16.585, 16.79, 16.805, 16.985, 17.03, 17.285, 17.351, 17.55, 17.585, 17.805, 17.91, 18.11

In [ ]:
import concurrent.futures
import datetime
import functools
import math
import os
from io import BytesIO

import fastf1
import numpy as np
import pandas as pd
import requests
import streamlit as st
from fastapi import Depends, FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse, HTMLResponse
from fastf1.ergast import Ergast
from pydantic import BaseModel, Field
from sqlalchemy.orm import Session

from . import accelerations, database, models, utils

# import accelerations
# import database
# import models
# import utils
FASTF1_CACHE_DIR = os.environ["FASTF1_CACHE_DIR"]

fastf1.Cache.enable_cache(FASTF1_CACHE_DIR)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


database.Base.metadata.create_all(bind=database.engine)


def get_db():
    try:
        db = database.SessionLocal()
        yield db
    finally:
        db.close()


class RacePace(BaseModel):
    year: int
    event: str
    session: str
    Driver: str
    LapTime: float
    Diff: float
    Team: str
    fill: str


# @functools.cache
@app.get("/racepace/{year}/{event}/{session}", response_model=None)
async def average_race_pace(
    year: int, event: str | int, session: str, db: Session = Depends(get_db)
) -> any:
    race_pace_data = (
        db.query(models.RacePace)
        .filter_by(year=year, event=event, session=session)
        .all()
    )

    if race_pace_data:
        print("Fetching from Database")

    if not race_pace_data:
        print("Writing to Database")
        f1session = fastf1.get_session(
            year,
            event,
            session,
            # backend="fastf1",
            # force_ergast=False,
        )
        f1session.load(telemetry=False, weather=False, messages=False)
        laps = f1session.laps

        laps = laps.loc[laps.LapNumber > 1]
        laps = laps.pick_track_status(
            "1",
        )
        laps["LapTime"] = laps.Sector1Time + laps.Sector2Time + laps.Sector3Time

        # convert LapTime to seconds
        laps["LapTime"] = laps["LapTime"].apply(lambda x: x.total_seconds())

        laps = laps.loc[laps.LapTime < laps.LapTime.min() * 1.07]

        df = (
            laps[["LapTime", "Driver"]].groupby("Driver").mean().reset_index(drop=False)
        )
        df = df.sort_values(by="LapTime").reset_index(drop=True)
        df["LapTime"] = df["LapTime"].round(3)
        df["Diff"] = (df["LapTime"] - df["LapTime"].min()).round(3)
        teams = laps[["Driver", "Team"]].drop_duplicates().reset_index(drop=True)
        # join teams and df
        df = df.merge(teams, on="Driver", how="left")

        car_colors = utils.team_colors(year)

        df["fill"] = df["Team"].map(car_colors)

        df_json = df.to_dict("records")

        # save the data to the database
        for record in df.to_dict("records"):
            race_pace = models.RacePace(**record)
            db.add(race_pace)

        db.commit()

        return {"racePace": df_json}

    return {"racePace": [dict(race_pace) for race_pace in race_pace_data]}


@functools.cache
@app.get("/topspeed/{year}/{event}/{session}", response_model=None)
async def top_speed(year: int, event: str | int, session: str) -> any:
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps
    team_colors = utils.team_colors(year)

    fastest_speedtrap = (
        laps[["SpeedI1", "SpeedI2", "SpeedST", "SpeedFL"]]
        .idxmax(axis=1)
        .value_counts()
        .index[0]
    )

    speed_df = (
        laps[[fastest_speedtrap, "Driver", "Compound", "Team"]]
        .groupby("Driver")
        .max()
        .sort_values(fastest_speedtrap, ascending=False)
        .reset_index()
    )
    # add team colors to dataframe
    speed_df["fill"] = speed_df["Team"].apply(lambda x: team_colors[x])

    # rename fastest speedtrap column to TopSpeed
    speed_df.rename(columns={fastest_speedtrap: "TopSpeed"}, inplace=True)

    # remove nan values in any column
    speed_df = speed_df.dropna()

    # Convert to int
    speed_df["TopSpeed"] = speed_df["TopSpeed"].astype(int)

    speed_dict = speed_df.to_dict(orient="records")

    return {"topSpeed": speed_dict}


@functools.cache
@app.get("/overtakes/{year}/{event}", response_model=None)
def get_overtakes(year: int, event: str) -> any:
    def get_overtakes_df(year, event):
        if year == 2023:
            url = "https://docs.google.com/spreadsheets/d/1M4aepPJaIfdqE9oU3L-2CQqKIyubLXG4Q4cqWnyqxp4/export?format=csv"
        if year == 2022:
            url = "https://docs.google.com/spreadsheets/d/1cuS3B6hk4iQmMaRQoMTcogIInJpavnV7rKuEsiJnEbU/export?format=csv"
        if year == 2021:
            url = "https://docs.google.com/spreadsheets/d/1ANQnPVkefRmvzrmGvEqXoqQ4dBfgcI_R9FPg-0BcM34/export?format=csv"
        if year == 2020:
            url = "https://docs.google.com/spreadsheets/d/1eG9WTkXKzFT4NMh-WqHOMs5G0UuPGnb6wP4CnFD8uzY/export?format=csv"
        if year == 2019:
            url = "https://docs.google.com/spreadsheets/d/10nHg7BIs5ySh_dE9uuIz2lq-gRWcg02tIMr0EPgPvJs/export?format=csv"
        if year == 2018:
            url = "https://docs.google.com/spreadsheets/d/1MyAwQdczccdca_FAIiZKkqZNauNh3ts99JZ278S2OKc/export?format=csv"

        response = requests.get(url, timeout=10)
        df = pd.read_csv(BytesIO(response.content))
        df = df[["Driver", event]]
        # replace NaNs with 0s
        df = df.fillna(0)
        # convert numbers to ints
        df[event] = df[event].astype(int)
        # replace event with "overtakes"
        df = df.rename(columns={event: "overtakes"})
        return df

    def get_overtaken_df(year, event):
        if year == 2023:
            url = "https://docs.google.com/spreadsheets/d/1wszzx694Ot-mvA5YrFCpy3or37xMgnC0XpE8uNnJLWk/export?format=csv"
        if year == 2022:
            url = "https://docs.google.com/spreadsheets/d/19_XFDD3BZDIQVkNE4bG6dwuKvMaO4g5HNaUARGaJwhE/export?format=csv"
        if year == 2021:
            url = "https://docs.google.com/spreadsheets/d/1dQBHnd3AXEPNH5I75cjbzAAzi9ipqGk3v9eZT9eYKS4/export?format=csv"
        if year == 2020:
            url = "https://docs.google.com/spreadsheets/d/1snyntPMxYH4_KHSRI96AwBoJQrPbX6OanJAcqbYyW-Y/export?format=csv"
        if year == 2019:
            url = "https://docs.google.com/spreadsheets/d/11FfFkXErJg7F22iVwJo9XfLFAWucMBVlzL1qUGWxM3s/export?format=csv"
        if year == 2018:
            url = "https://docs.google.com/spreadsheets/d/1XJXAEyRpRS_UwLHzEtN2PdIaFJYGWSN6ypYN8Ecwp9A/export?format=csv"

        response = requests.get(url, timeout=10)
        df = pd.read_csv(BytesIO(response.content))
        df = df[["Driver", event]]
        # replace NaNs with 0s
        df = df.fillna(0)
        # convert numbers to ints
        df[event] = df[event].astype(int)
        df = df.rename(columns={event: "overtaken"})
        return df

    overtakes = get_overtakes_df(year, event)
    overtaken = get_overtaken_df(year, event)
    df = overtakes.merge(overtaken, on="Driver")

    # remove drivers with 0 overtakes and 0 overtaken
    df = df[(df["overtakes"] != 0) | (df["overtaken"] != 0)]

    # sort in the decreasing order of overtakes
    df = df.sort_values(
        by=["overtakes", "overtaken"], ascending=[False, True]
    ).reset_index(drop=True)
    # convert to dictionary
    df_dict = df.to_dict(orient="records")

    return {"overtakes": df_dict}


@functools.cache
@app.get("/fastest/{year}/{event}/{session}", response_model=None)
async def fastest_lap(year: int, event: str | int, session: str) -> any:
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps

    drivers = pd.unique(laps["Driver"])

    list_fastest_laps = list()

    for drv in drivers:
        drvs_fastest_lap = laps.pick_driver(drv).pick_fastest()
        list_fastest_laps.append(drvs_fastest_lap)

    df = (
        fastf1.core.Laps(list_fastest_laps)
        .sort_values(by="LapTime")
        .reset_index(drop=True)
    )

    pole_lap = df.pick_fastest()
    df["Diff"] = df["LapTime"] - pole_lap["LapTime"]

    car_colors = utils.team_colors(year)

    df["fill"] = df["Team"].map(car_colors)

    # convert timedelta to float and round to 3 decimal places
    df["Diff"] = df["Diff"].dt.total_seconds().round(3)
    df = df[["Driver", "LapTime", "Diff", "Team", "fill"]]

    # remove nan values in any column
    df = df.dropna()

    df_json = df.to_dict("records")

    return {"fastest": df_json}


# @st.cache_data


@app.get("/wdc", response_model=None)
async def driver_standings() -> any:
    YEAR = 2023  # datetime.datetime.now().year
    df = pd.DataFrame(
        pd.read_html(f"https://www.formula1.com/en/results.html/{YEAR}/drivers.html")[0]
    )
    df = df[["Driver", "PTS", "Car"]]
    # reverse the order
    df = df.sort_values(by="PTS", ascending=True)

    # in Driver column only keep the last 3 characters
    df["Driver"] = df["Driver"].str[:-5]

    # add colors to the dataframe
    car_colors = utils.team_colors(YEAR)
    df["fill"] = df["Car"].map(car_colors)

    # remove rows where points is 0
    df = df[df["PTS"] != 0]
    df.reset_index(inplace=True, drop=True)
    df.rename(columns={"PTS": "Points"}, inplace=True)

    return {"WDC": df.to_dict("records")}


# @st.cache_data


@app.get("/", response_model=None)
async def root():
    return HTMLResponse(
        content="""<iframe src="https://tracinginsights-f1-analysis.hf.space" frameborder="0" style="width:100%; height:100%;" scrolling="yes" allowfullscreen:"yes"></iframe>""",
        status_code=200,
    )


# @st.cache_data


@app.get("/years", response_model=None)
async def years_available() -> any:
    # make a list from 2018 to current year
    current_year = datetime.datetime.now().year
    years = list(range(2018, current_year + 1))
    # reverse the list to get the latest year first
    years.reverse()
    years = [{"label": str(year), "value": year} for year in years]
    return {"years": years}


# format for events {"events":[{"label":"Saudi Arabian Grand Prix","value":2},{"label":"Bahrain Grand Prix","value":1},{"label":"Pre-Season Testing","value":"t1"}]}

# @st.cache_data


@app.get("/{year}", response_model=None)
async def events_available(year: int) -> any:
    # get events available for a given year
    data = utils.LatestData(year)
    events = data.get_events()
    events = [{"label": event, "value": event} for i, event in enumerate(events)]
    events.reverse()

    return {"events": events}


# format for sessions {"sessions":[{"label":"FP1","value":"FP1"},{"label":"FP2","value":"FP2"},{"label":"FP3","value":"FP3"},{"label":"Qualifying","value":"Q"},{"label":"Race","value":"R"}]}


# @st.cache_data
@functools.cache
@app.get("/{year}/{event}", response_model=None)
async def sessions_available(year: int, event: str | int) -> any:
    # get sessions available for a given year and event
    data = utils.LatestData(year)
    sessions = data.get_sessions(event)
    sessions = [{"label": session, "value": session} for session in sessions]

    return {"sessions": sessions}


# format for drivers {"drivers":[{"color":"#fff500","label":"RIC","value":"RIC"},{"color":"#ff8700","label":"NOR","value":"NOR"},{"color":"#c00000","label":"VET","value":"VET"},{"color":"#0082fa","label":"LAT","value":"LAT"},{"color":"#787878","label":"GRO","value":"GRO"},{"color":"#ffffff","label":"GAS","value":"GAS"},{"color":"#f596c8","label":"STR","value":"STR"},{"color":"#787878","label":"MAG","value":"MAG"},{"color":"#0600ef","label":"ALB","value":"ALB"},{"color":"#ffffff","label":"KVY","value":"KVY"},{"color":"#fff500","label":"OCO","value":"OCO"},{"color":"#0600ef","label":"VER","value":"VER"},{"color":"#00d2be","label":"HAM","value":"HAM"},{"color":"#ff8700","label":"SAI","value":"SAI"},{"color":"#00d2be","label":"BOT","value":"BOT"},{"color":"#960000","label":"GIO","value":"GIO"}]}

# @st.cache_data


@functools.cache
@app.get("/strategy/{year}/{event}", response_model=None)
async def get_strategy(year: int, event: str | int) -> any:
    f1session = fastf1.get_session(year, event, "R")
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps

    drivers_list = pd.unique(laps["Driver"])

    drivers = pd.DataFrame(drivers_list, columns=["Driver"])
    drivers["FinishOrder"] = drivers.index + 1

    # Get the LapNumber of the first lap of each stint
    first_lap = (
        laps[["Driver", "Stint", "Compound", "LapNumber"]]
        .groupby(["Driver", "Stint", "Compound"])
        .first()
        .reset_index()
    )
    #  Add FinishOrder to first_lap
    first_lap = pd.merge(first_lap, drivers, on="Driver")
    # change LapNumber to LapStart
    first_lap = first_lap.rename(columns={"LapNumber": "LapStart"})
    # reduce the lapstart by 1
    first_lap["LapStart"] = first_lap["LapStart"] - 1

    # find the last lap of each stint
    last_lap = (
        laps[["Driver", "Stint", "Compound", "LapNumber"]]
        .groupby(["Driver", "Stint", "Compound"])
        .last()
        .reset_index()
    )
    #  change LapNumber to LapEnd
    last_lap = last_lap.rename(columns={"LapNumber": "LapEnd"})

    # combine first_lap and last_lap
    stint_laps = pd.merge(first_lap, last_lap, on=["Driver", "Stint", "Compound"])
    #  to cover for outliers
    stint_laps["fill"] = "white"

    stint_laps["fill"] = stint_laps["Compound"].map(
        {
            "SOFT": "red",
            "MEDIUM": "yellow",
            "HARD": "white",
            "INTERMEDIATE": "blue",
            "WET": "green",
        }
    )

    # sort by FinishOrder
    stint_laps = stint_laps.sort_values(by=["FinishOrder"], ascending=[True])

    stint_laps_dict = stint_laps.to_dict("records")

    return {"strategy": stint_laps_dict}

# lap position
@functools.cache
@app.get("/lapchart/{year}/{event}/{session}", response_model=None)
async def lap_chart(
    year: int,
    event: str | int,
    session: str,
) -> any:
    ergast = Ergast()

    race_names_df = ergast.get_race_schedule(season=year, result_type="pandas")
    event_number = race_names_df[race_names_df["raceName"] == event]["round"].values[0]
    drivers_df = ergast.get_driver_info(
        season=year, round=event_number, result_type="pandas"
    )
    laptimes_df = ergast.get_lap_times(
        season=year, round=event_number, result_type="pandas", limit=2000
    ).content[0]
    laptimes_df = pd.merge(laptimes_df, drivers_df, how="left", on="driverId")

    results_df = ergast.get_race_results(
        season=year, round=event_number, result_type="pandas"
    ).content[0]
    results_df = results_df[["driverCode", "constructorName"]]

    # merge results_df on laptime_df
    laptimes_df = pd.merge(laptimes_df, results_df, how="left", on="driverCode")

    team_colors = utils.team_colors(year)
    # add team_colors to laptimes_df
    laptimes_df["fill"] = laptimes_df["constructorName"].map(team_colors)

    #  rename number as x and position as y
    laptimes_df.rename(
        columns={"number": "x", "position": "y", "driverCode": "id"}, inplace=True
    )

    lap_chart_data = []

    for driver in laptimes_df["id"].unique():
        data = laptimes_df[laptimes_df["id"] == driver]
        fill = data["fill"].values[0]
        data = data[["x", "y"]]
        data_dict = data.to_dict(orient="records")
        driver_dict = {"id": driver, "fill": fill, "data": data_dict}
        # add this to all_data
        lap_chart_data.append(driver_dict)

    lap_chart_dict = {"lapChartData": lap_chart_data}

    return lap_chart_dict


@functools.cache
@app.get("/{year}/{event}/{session}", response_model=None)
async def session_drivers(year: int, event: str | int, session: str) -> any:
    # get drivers available for a given year, event and session
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)

    laps = f1session.laps
    team_colors = utils.team_colors(year)
    # add team_colors dict to laps on Team column
    laps["color"] = laps["Team"].map(team_colors)

    unique_drivers = laps["Driver"].unique()

    drivers = [
        {
            "color": laps[laps.Driver == driver].color.iloc[0],
            "label": driver,
            "value": driver,
        }
        for driver in unique_drivers
    ]

    return {"drivers": drivers}

# select laps similar to select drivers select box
@functools.cache
@app.get("/laps/{year}/{event}/{session}", response_model=None)
async def get_driver_laps_data(year: int, event: str | int, session: str) -> any:
    # get drivers available for a given year, event and session
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps
    team_colors = utils.team_colors(year)
    # add team_colors dict to laps on Team column
    laps["color"] = laps["Team"].map(team_colors)

    # combine Driver and LapNumber as a new column
    laps["label"] = (
        laps["Driver"]
        + "-"
        + laps["LapNumber"].astype(int).astype(str)
        + "-"
        + str(year)
        + "-"
        + event
        + "-"
        + session
    )
    laps["value"] = (
        laps["Driver"]
        + "-"
        + laps["LapNumber"].astype(int).astype(str)
        + "-"
        + str(year)
        + "-"
        + event
        + "-"
        + session
    )

    laps = laps[["value", "label", "color"]]

    driver_laps_dict = laps.to_dict("records")

    return {"laps": driver_laps_dict}


# format for chartData {"chartData":[{"lapnumber":1},{
# "VER":91.564,
# "VER_compound":"SOFT",
# "VER_compound_color":"#FF5733",
# "lapnumber":2
# },{"lapnumber":3},{"VER":90.494,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":4},{"lapnumber":5},{"VER":90.062,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":6},{"lapnumber":7},{"VER":89.815,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":8},{"VER":105.248,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":9},{"lapnumber":10},{"VER":89.79,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":11},{"VER":145.101,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":12},{"lapnumber":13},{"VER":89.662,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":14},{"lapnumber":15},{"VER":89.617,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":16},{"lapnumber":17},{"VER":140.717,"VER_compound":"SOFT","VER_compound_color":"#FF5733","lapnumber":18}]}

# @st.cache_data


@functools.cache
@app.get("/{year}/{event}/{session}/{driver}", response_model=None)
async def laps_data(year: int, event: str | int, session: str, driver: str) -> any:
    # get drivers available for a given year, event and session
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps
    team_colors = utils.team_colors(year)
    # add team_colors dict to laps on Team column

    drivers = laps.Driver.unique()
    # for each driver in drivers, get the Team column from laps and get the color from team_colors dict
    drivers = [
        {
            "color": team_colors[laps[laps.Driver == driver].Team.iloc[0]],
            "label": driver,
            "value": driver,
        }
        for driver in drivers
    ]

    driver_laps = laps.pick_driver(driver)
    driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()
    # remove rows where LapTime is null
    driver_laps = driver_laps[driver_laps.LapTime.notnull()]
    compound_colors = {
        "SOFT": "#FF0000",
        "MEDIUM": "#FFFF00",
        "HARD": "#FFFFFF",
        "INTERMEDIATE": "#00FF00",
        "WET": "#088cd0",
    }

    driver_laps_data = []

    for _, row in driver_laps.iterrows():
        if row["LapTime"] > 0:
            lap = {
                f"{driver}": row["LapTime"],
                f"{driver}_compound": row["Compound"],
                f"{driver}_compound_color": compound_colors[row["Compound"]],
                "lapnumber": row["LapNumber"],
            }
        else:
            lap = {"lapnumber": row["LapNumber"]}

        driver_laps_data.append(lap)

    return {"chartData": driver_laps_data}


@functools.cache
@app.get("/laptimes/{year}/{event}/{session}/{driver}", response_model=None)
async def get_laps_data(year: int, event: str | int, session: str, driver: str) -> any:
    # get drivers available for a given year, event and session
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=False, weather=False, messages=False)
    laps = f1session.laps
    team_colors = utils.team_colors(year)
    # add team_colors dict to laps on Team column

    drivers = laps.Driver.unique()
    # for each driver in drivers, get the Team column from laps and get the color from team_colors dict
    drivers = [
        {
            "color": team_colors[laps[laps.Driver == driver].Team.iloc[0]],
            "label": driver,
            "value": driver,
        }
        for driver in drivers
    ]

    driver_laps = laps.pick_driver(driver)
    driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()
    driver_laps = driver_laps[["Driver", "LapTime", "LapNumber", "Compound"]]

    # remove rows where LapTime is null
    driver_laps = driver_laps[driver_laps.LapTime.notnull()]

    driver_laps_dict = driver_laps.to_dict("records")

    return {"chartData": driver_laps_dict}


# @st.cache_data


@functools.cache
@app.get("/{year}/{event}/{session}/{driver}/{lap_number}", response_model=None)
async def telemetry_data(
    year: int, event: str | int, session: str, driver: str, lap_number: int
) -> any:
    f1session = fastf1.get_session(year, event, session)
    f1session.load(telemetry=True, weather=False, messages=False)
    laps = f1session.laps

    driver_laps = laps.pick_driver(driver)
    driver_laps["LapTime"] = driver_laps["LapTime"].dt.total_seconds()

    # get the telemetry for lap_number
    selected_lap = driver_laps[driver_laps.LapNumber == lap_number]

    telemetry = selected_lap.get_telemetry()

    lon_acc, lat_acc = accelerations.compute_accelerations(telemetry)
    telemetry["lon_acc"] = lon_acc
    telemetry["lat_acc"] = lat_acc

    telemetry["Time"] = telemetry["Time"].dt.total_seconds()

    laptime = selected_lap.LapTime.values[0]
    data_key = f"{driver} - Lap {int(lap_number)} - {year} {session} [laptime]"

    telemetry["DRS"] = telemetry["DRS"].apply(lambda x: 1 if x in [10, 12, 14] else 0)

    brake_tel = []
    drs_tel = []
    gear_tel = []
    rpm_tel = []
    speed_tel = []
    throttle_tel = []
    time_tel = []
    track_map = []
    lon_acc_tel = []
    lat_acc_tel = []

    for _, row in telemetry.iterrows():
        brake = {
            "x": row["Distance"],
            "y": row["Brake"],
        }
        brake_tel.append(brake)

        drs = {
            "x": row["Distance"],
            "y": row["DRS"],
        }
        drs_tel.append(drs)

        gear = {
            "x": row["Distance"],
            "y": row["nGear"],
        }
        gear_tel.append(gear)

        rpm = {
            "x": row["Distance"],
            "y": row["RPM"],
        }
        rpm_tel.append(rpm)

        speed = {
            "x": row["Distance"],
            "y": row["Speed"],
        }
        speed_tel.append(speed)

        throttle = {
            "x": row["Distance"],
            "y": row["Throttle"],
        }
        throttle_tel.append(throttle)

        time = {
            "x": row["Distance"],
            "y": row["Time"],
        }
        time_tel.append(time)

        lon_acc = {
            "x": row["Distance"],
            "y": row["lon_acc"],
        }
        lon_acc_tel.append(lon_acc)

        lat_acc = {
            "x": row["Distance"],
            "y": row["lat_acc"],
        }
        lat_acc_tel.append(lat_acc)

        track = {
            "x": row["X"],
            "y": row["Y"],
        }
        track_map.append(track)

    telemetry_data = {
        "telemetryData": {
            "brake": brake_tel,
            "dataKey": data_key,
            "drs": drs_tel,
            "gear": gear_tel,
            "rpm": rpm_tel,
            "speed": speed_tel,
            "throttle": throttle_tel,
            "time": time_tel,
            "lon_acc": lon_acc_tel,
            "lat_acc": lat_acc_tel,
            "trackMap": track_map,
        }
    }

    return telemetry_data